In [1]:
from data_preprocess import read_and_normalize_train_data, read_and_normalize_test_data
import numpy as np
import time
from vgg16_run import save_model, read_model

/home/alpha/anaconda3/envs/tensorflow1.8/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/alpha/anaconda3/envs/tensorflow1.8/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
img_rows, img_cols = 224, 224
batch_size = 32
nb_epoch = 2

In [3]:
x_train, y_train, driver_id, unique_drivers = read_and_normalize_train_data(img_rows, img_cols, color_type=3)

Restore train from cache!


In [4]:
x_train = np.transpose(x_train, [0,2,3,1])

In [5]:
print(x_train.shape, x_train.nbytes)
print(y_train.shape, y_train.nbytes)

(22424, 224, 224, 3) 13501759488
(22424, 10) 896960


In [6]:
import tensorflow as tf
from keras.utils import plot_model
from keras.applications import ResNet50
from keras.models import Model
from keras.layers import Dense, Flatten

In [7]:
def resnet50_model():
    base_model = ResNet50(include_top=False, input_shape=(224,224,3))
    resnet50_output = base_model.output
    predictions = Dense(10, activation='softmax')(Flatten()(resnet50_output))
    model = Model(inputs=base_model.input, outputs=predictions)
    
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
    return model

In [8]:
from sklearn.model_selection import KFold

In [3]:
model_name='resnet50'

In [9]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [10]:
num_fold = 0

for train_index, test_index in kf.split(x_train):
    num_fold += 1
    x = x_train[train_index]
    y = y_train[train_index]
    
    model = resnet50_model()
    model.fit(x, y, batch_size=batch_size, epochs=nb_epoch,
             verbose=1, validation_split=0.2, shuffle=True)
    
    save_model(model, num_fold, model_name)    

Train on 16144 samples, validate on 4037 samples
Epoch 1/2
16144/16144 [==============================] - 58s 4ms/step - loss: 1.2970 - val_loss: 8.4777
Epoch 2/2
16144/16144 [==============================] - 57s 4ms/step - loss: 0.7305 - val_loss: 9.3233
Train on 16144 samples, validate on 4037 samples
Epoch 1/2
16144/16144 [==============================] - 60s 4ms/step - loss: 1.2881 - val_loss: 7.6468
Epoch 2/2
16144/16144 [==============================] - 59s 4ms/step - loss: 0.7330 - val_loss: 8.8532
Train on 16144 samples, validate on 4037 samples
Epoch 1/2
16144/16144 [==============================] - 58s 4ms/step - loss: 1.3125 - val_loss: 9.9864
Epoch 2/2
16144/16144 [==============================] - 56s 3ms/step - loss: 0.7407 - val_loss: 10.7227
Train on 16144 samples, validate on 4037 samples
Epoch 1/2
16144/16144 [==============================] - 58s 4ms/step - loss: 1.2988 - val_loss: 8.9471
Epoch 2/2
16144/16144 [==============================] - 55s 3ms/step - los

In [13]:
del x
del y

NameError: name 'x' is not defined

In [ ]:
num_fold=11
from vgg16_run import read_model, merge_several_folds_mean
print('Start testing............')
test_data, test_id = read_and_normalize_test_data(img_rows, img_cols, 3)
yfull_test = []

for index in range(1, num_fold + 1):
    # Store test predictions
    model = read_model(index, model_name)
    test_prediction = model.predict(test_data, batch_size=128, verbose=1)
    yfull_test.append(test_prediction)

info_string = 'loss_' + modelStr \
    + '_r_' + str(img_rows) \
    + '_c_' + str(img_cols) \
    + '_folds_' + str(nfolds) \
    + '_ep_' + str(nb_epoch)

test_res = merge_several_folds_mean(yfull_test, nfolds)

Start testing............


  4%|▍         | 3325/79726 [00:09<03:49, 332.57it/s]/home/alpha/anaconda3/envs/tensorflow1.8/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
 90%|█████████ | 71961/79726 [04:39<00:30, 257.41it/s]

[ 3  4  6  7 10 14 15 17 19 20 22 23]
[ 0  1  2  5  8  9 11 12 13 16 18 21]
